In [ ]:
#İndirilmesi gereken kütüphaneler
!pip install simpletransformers

In [ ]:
#Data setini yüklemek için drive'ı içe aktarma
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import nltk
import json

# NLTK paketini indir (Gerekli NLTK kaynaklarını indirir)
nltk.download('punkt')

# CSV dosyasını oku (Veri setini CSV dosyasından okur)
df = pd.read_csv("/content/sonHali.csv")

# Yalnızca 'text', 'kullanım_label' ve 'hizmet_label' sütunlarını kullan (Gereksiz sütunları çıkarır)
df = df[['text', 'kullanım_label', 'hizmet_label']]

# JSON formatında veriyi hazırlamak (Veriyi JSON formatına uygun hale getirir)
data = {
    "Text": df['text'].tolist(),  # Tüm metinleri liste olarak ekler
    "Entities": []  # Her bir metin için etiketli varlıkları tutacak boş bir liste oluşturur
}

# Her satırdaki veriyi işler
for index, row in df.iterrows():
    entity_list = []  # Bu satır için varlıkları tutacak liste

    # kullanım_label sütunundaki etiketleri ekle
    if not pd.isna(row['kullanım_label']):
        kullanım_entities = json.loads(row['kullanım_label'])  # JSON stringini sözlüğe çevirir
        for entity in kullanım_entities:
            entity_list.append({
                "start": entity["start"],  # Varlık başlangıç indexi
                "end": entity["end"],  # Varlık bitiş indexi
                "text": entity["text"],  # Varlığın metni
                "labels": entity["labels"]  # Varlığın etiketleri
            })

    # hizmet_label sütunundaki etiketleri ekle
    if not pd.isna(row['hizmet_label']):
        hizmet_entities = json.loads(row['hizmet_label'])  # JSON stringini sözlüğe çevirir
        for entity in hizmet_entities:
            entity_list.append({
                "start": entity["start"],  # Varlık başlangıç indexi
                "end": entity["end"],  # Varlık bitiş indexi
                "text": entity["text"],  # Varlığın metni
                "labels": entity["labels"]  # Varlığın etiketleri
            })

    # Tüm varlıkları bu metin için listeye ekler
    data["Entities"].append(entity_list)

# JSON dosyasını kaydet (Veriyi JSON dosyasına yazar)
output_json_path = "output_son.json"
with open(output_json_path, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print("JSON dosyası başarıyla oluşturuldu.")

# JSON dosyasını aç ve içeriği bir değişkene ata (JSON dosyasını okur ve veriyi yükler)
with open(output_json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Veriyi bir DataFrame'e yükle (JSON verisini pandas DataFrame formatına çevirir)
df = pd.DataFrame({
    "Text": data["Text"],  # Metin sütunu
    "Entities": data["Entities"]  # Varlıklar sütunu
})

# Metinleri tokenize edip varlıkları etiketlemek için bir fonksiyon
def tokenize_and_annotate(df):
    tokens = []  # Token'ları tutacak liste
    labels = []  # Etiketleri tutacak liste
    for _, row in df.iterrows():
        tweet = row['Text']
        entities = row['Entities']

        # Metnin string formatında olduğundan emin olun
        if not isinstance(tweet, str):
            tweet = str(tweet)

        # Metni tokenize et
        tokenized_text = nltk.word_tokenize(tweet)
        token_labels = ['O'] * len(tokenized_text)  # Tüm token'lar için varsayılan etiket 'O'

        # Her varlık için
        for entity in entities:
            start_idx = entity['start']  # Varlık başlangıç indeksi
            end_idx = entity['end']  # Varlık bitiş indeksi
            entity_text = entity['labels'][0]  # İlk etiketi alır (Varsayılan olarak bir etiket olduğu varsayılır)

            # Varlık için token indekslerini bul
            char_idx = 0
            token_start = -1
            token_end = -1
            for i, token in enumerate(tokenized_text):
                char_idx = tweet.find(token, char_idx)
                if char_idx == start_idx:
                    token_start = i
                if char_idx + len(token) >= end_idx:
                    token_end = i
                    break
                char_idx += len(token)

            # Eğer varlık bulunmuşsa, etiketleri atar
            if token_start != -1 and token_end != -1:
                token_labels[token_start] = f'B-{entity_text}'  # Varlığın başlangıç etiketi
                for i in range(token_start + 1, token_end + 1):
                    token_labels[i] = f'I-{entity_text}'  # Varlığın iç etiketi

        tokens.append(tokenized_text)
        labels.append(token_labels)

    return tokens, labels

# Metinleri tokenize et ve varlıkları etiketle
tokens, labels = tokenize_and_annotate(df)

# Token'lar ve etiketler ile bir DataFrame oluştur
annotated_df = pd.DataFrame({
    'Tokens': tokens,  # Token sütunu
    'Labels': labels  # Etiket sütunu
})

# DataFrame'i bir Excel dosyasına kaydet
output_excel_path = 'data.xlsx'
annotated_df.to_excel(output_excel_path, index=False)

print(f"Tokenized and annotated dataset has been saved to {output_excel_path}")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


JSON dosyası başarıyla oluşturuldu.
Tokenized and annotated dataset has been saved to data.xlsx


In [ ]:
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

# 1. Veriyi Yükle ve Hazırla
# Excel dosyasından veriyi yükler
df = pd.read_excel('/content/data.xlsx')

# String olarak kaydedilen liste temsillerini gerçek Python listelerine dönüştürür
df['Tokens'] = df['Tokens'].apply(eval)
df['Labels'] = df['Labels'].apply(eval)

# SimpleTransformers formatına uygun hale getirir
data = []
sentence_id = 0  # Cümle ID'si için başlangıç değeri
for tokens, labels in zip(df['Tokens'], df['Labels']):
    for token, label in zip(tokens, labels):
        data.append([sentence_id, token, label])  # Her bir token ve etiketi bir liste olarak ekler
    sentence_id += 1  # Bir sonraki cümle için cümle ID'sini artırır

# Hazırlanan veriyi bir DataFrame'e dönüştürür
data_df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

# Tüm etiketlerin listesini çıkarır (benzersiz etiketler)
labels_list = list(set([label for labels in df['Labels'] for label in labels]))

# 2. Modeli Konfigüre Et ve Eğit
# Modelin eğitim ayarlarını yapar
model_args = NERArgs()
model_args.num_train_epochs = 10  # Eğitim epoch sayısını belirler
model_args.train_batch_size = 16  # Eğitim batch boyutunu belirler
model_args.eval_batch_size = 16  # Değerlendirme batch boyutunu belirler
model_args.learning_rate = 3e-5  # Öğrenme oranını belirler
model_args.save_steps = -1  # Adım başına kaydetmeyi devre dışı bırakır
model_args.save_model_every_epoch = False  # Her epoch sonunda modeli kaydetmeyi devre dışı bırakır
model_args.overwrite_output_dir = True  # Çıktı dizinini üzerine yazmak için ayar
model_args.labels_list = labels_list  # Etiket listesini model argümanlarına ekler

# BERT tabanlı bir Named Entity Recognition (NER) modelini oluşturur
model = NERModel(
    "bert",  # Model türü
    "dbmdz/bert-base-turkish-128k-uncased",  # Türkçe BERT modeli
    args=model_args,  # Eğitim ayarları
    use_cuda=True  # CUDA kullanarak eğitimi hızlandırmak için
)

# Modeli eğitim verisi üzerinde eğitir
model.train_model(data_df)

# 3. Modeli Değerlendir
# Eğitilmiş modeli veri seti üzerinde değerlendirir
result, model_outputs, preds_list = model.eval_model(data_df)
print(result)  # Değerlendirme sonuçlarını yazdırır

# 4. Yeni Cümleler Üzerinde Tahmin Yap
# Yeni bir cümle üzerinde modelin tahmin yapmasını sağlar
predictions, raw_outputs = model.predict(["Bu ürünü çok beğendim, ses kalitesi harika, paketlemesi çok kötüydü"])
print(predictions)  # Tahmin edilen varlıkları ve etiketleri yazdırır


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/320 [00:00<?, ?it/s]

{'eval_loss': 0.09417395973869133, 'precision': 0.8773657149108413, 'recall': 0.9135436837908646, 'f1_score': 0.8950892857142856}


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Bu': 'O'}, {'ürünü': 'O'}, {'çok': 'O'}, {'beğendim,': 'O'}, {'ses': 'B-Ses kalitesi'}, {'kalitesi': 'I-Ses kalitesi'}, {'harika,': 'O'}, {'paketlemesi': 'B-Teslimat ve Paketleme'}, {'çok': 'O'}, {'kötüydü': 'O'}]]


In [ ]:
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# 1. CSV dosyasını oku
# Veriyi CSV dosyasından yükler
df = pd.read_csv("/content/sonHali.csv")

# 2. Veriyi uygun formata getir
data = []
for _, row in df.iterrows():
    # Her satır için entity listesi başlatılır
    entities = []

    # NaN (eksik) değilse, değeri string olarak alır
    if not pd.isna(row['duygu_label']):
        # JSON objesini yükler
        try:
            entities = json.loads(str(row['duygu_label']))
        except json.JSONDecodeError:
            # JSON hatası durumunda hatayı bildirir ve devam eder
            print(f"JSON decode error for row: {row['duygu_label']}")
            continue

    # Her entity üzerinde iterasyon yapar ve veriyi listeye ekler
    for entity in entities:
        # Entity geçerli bir dict ise ve 'text' ve 'labels' anahtarlarını içeriyorsa
        if isinstance(entity, dict) and 'text' in entity and 'labels' in entity:
            data.append({
                "text": entity["text"],
                "labels": entity["labels"][0]  # Bir etiket olduğunu varsayar
            })
        else:
            # Geçersiz entity durumunda hatayı bildirir
            print(f"Skipping invalid entity: {entity}")

# 3. Eğitim verisini DataFrame'e dönüştür
# Hazırlanan veriyi bir DataFrame'e dönüştürür
train_df = pd.DataFrame(data)

# 4. Veriyi eğitim ve doğrulama setlerine böl
# Veriyi %90 eğitim ve %10 doğrulama seti olarak böler
train_data, eval_data = train_test_split(train_df, test_size=0.1, random_state=42)

# 5. Etiketlerin listesini çıkar
# Etiketleri liste olarak tanımlar ve bunlara karşılık gelen numaraları mapler
labels_list = ["negative", "neutral", "positive"]
labels_map = {"negative": 0, "neutral": 1, "positive": 2}

# 6. Model argümanlarını belirle
# Model için gerekli olan ayarları yapar
args = ClassificationArgs()
args.num_train_epochs = 10  # Eğitim epoch sayısını belirler
args.learning_rate = 3e-5  # Öğrenme oranını belirler
args.overwrite_output_dir = True  # Çıktı dizinini üzerine yazmayı etkinleştirir
args.train_batch_size = 16  # Eğitim batch boyutunu belirler
args.eval_batch_size = 16  # Değerlendirme batch boyutunu belirler
args.save_steps = -1  # Adım başına kaydetmeyi devre dışı bırakır
args.save_model_every_epoch = False  # Her epoch sonunda kaydetmeyi devre dışı bırakır
args.labels_list = labels_list  # Etiket listesini model argümanlarına ekler
args.output_dir = "sentiment_model_output"  # Çıktı modelinin kaydedileceği dizin
args.labels_map = None  # Etiket map'ini None olarak ayarlar

# 7. BERT tabanlı duygu analizi modelini oluştur
# BERT modelini oluşturur, Türkçe diline özel eğitimli versiyonu kullanır
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-128k-uncased', args=args, num_labels=len(labels_list))

# 8. Modeli eğit
# Modeli eğitim verisi üzerinde eğitir ve doğrulama setini kullanarak değerlendirme yapar
model.train_model(train_data, eval_df=eval_data)

# 9. Modeli kaydet
# Eğitilmiş modeli belirtilen dizine kaydeder
model.save_model("/content/sentiment_model_output")

# 10. Metrik fonksiyonlarını tanımla
# Precision, recall ve f1 skorlarını hesaplamak için metrik fonksiyonlarını tanımlar
def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

# 11. Doğrulama sonuçlarını al ve metrikleri hesapla
# Doğrulama verisi üzerinde modelin performansını değerlendirir ve metrikleri hesaplar
result, model_outputs, wrong_predictions = model.eval_model(eval_data, compute_metrics=compute_metrics)

# Sonuçları yazdır
print("Validation Results:", result)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/16 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/513 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/513 [00:00<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/513 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

Validation Results: {'mcc': 0.47638072179425095, 'compute_metrics': {'precision': 0.8650614603153582, 'recall': 0.8155872667398463, 'f1_score': 0.834305325938375}, 'eval_loss': 0.5455891620694545}


In [ ]:
# Google Drive'a dosya kopyalama işlemi
!cp /content/redmiBudsEssential_duzenlenmis.xlsx /content/drive/MyDrive

In [ ]:
from simpletransformers.ner import NERModel
from simpletransformers.classification import ClassificationModel

# 1. NER modelini yükle
ner_model = NERModel("bert", "/content/outputs", use_cuda=True)

# 2. Duygu analizi modelini yükle
sentiment_model = ClassificationModel("bert", "/content/sentiment_model_output", use_cuda=True)

# 3. Örnek cümle tanımla
sentence = "Bu ürünü çok beğendim, ses kalitesi harika, paketleme iyi değil"

# 4. Cümle üzerinde NER tahmini yap
ner_predictions, _ = ner_model.predict([sentence])

# 5. NER çıktısından varlıkları ve etiketlerini çıkar
entities = []
current_entity = []
current_label = None

# Her bir token üzerinde döngü yaparak varlıkları ve etiketleri topla
for token_info in ner_predictions[0]:
    for token, label in token_info.items():
        if label.startswith('B-') or (label.startswith('I-') and current_label != label[2:]):
            # Yeni bir varlık başlıyorsa, mevcut varlığı listeye ekle ve yeni varlığı başlat
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
                current_entity = []
            current_entity.append(token)
            current_label = label[2:]
        elif label.startswith('I-') and current_label == label[2:]:
            # Mevcut varlığın devamıysa, token'ı mevcut varlığa ekle
            current_entity.append(token)
        else:
            # Mevcut varlık sona erdiyse, listeye ekle ve devam et
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
                current_entity = []
            current_label = None

# Son varlığı ekle
if current_entity:
    entities.append((" ".join(current_entity), current_label))

# Cümleyi ve varlıkları yazdır
print(sentence)
print("Entities:")
for entity_text, label in entities:
    print(f"Entity: {entity_text}, Label: {label}")

# 6. Varlıklar üzerinde duygu analizi yap
entity_texts = [entity_text for entity_text, _ in entities]
sentiment_predictions, _ = sentiment_model.predict(entity_texts)

# 7. Sentiment predictions formatını kontrol et
print("Sentiment Predictions Format:")
print(sentiment_predictions)

# 8. Duygu analizi sonuçlarını yazdır
print("Sentiment Analysis:")
sentiment_labels = ["negative", "neutral", "positive"]
for entity_text, sentiment in zip(entity_texts, sentiment_predictions):
    print(f"Entity: {entity_text}, Sentiment: {sentiment}")


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Bu ürünü çok beğendim, ses kalitesi harika, paketleme iyi değil
Entities:
Entity: ses kalitesi, Label: Ses kalitesi
Entity: paketleme, Label: Teslimat ve Paketleme


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Sentiment Predictions Format:
['positive', 'positive']
Sentiment Analysis:
Entity: ses kalitesi, Sentiment: positive
Entity: paketleme, Sentiment: positive
